### Imports

In [1]:
import pickle
import numpy as np
import pandas as pd
import nltk

from corextopic import corextopic as ct
from nltk.corpus import stopwords

nltk.download('stopwords')

!pip install corextopic
!pip install networkx

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evanisenstein/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
wg_df = pickle.load(open('complete_df.pickle', 'rb'))
proba_df = pickle.load(open('proba_df.pickle', 'rb'))
vf_df = pickle.load(open('vf_df.pickle', 'rb'))

In [3]:
vf_df.head()

,Cassette Number,Conversation Number,Dates,Location_x,Minutes,Nixon Words,Participants,Full Dates,Full Transcript,files
0,E - 66,884-007,1973-03-20,White House Oval Office,4,"What I think, what it really gets down to, bas...","President Nixon; H. R. (""Bob"") Haldeman","MARCH 20, 1973",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,34
1,E - 67,885-007,1973-03-20,White House Oval Office,46,"Well, I'm trying to figure out with Ehrlichman...","President Nixon; H. R. (""Bob"") Haldeman","MARCH 20, 1973",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,35
2,E - 68,037-175,1973-03-20,White House Telephone,14,"John Dean, please. Hello. I hear you're havi...","President Nixon; John W. Dean, III; White Hous...","MARCH 20, 1973",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,36
3,E - 69,886-008,1973-03-21,White House Oval Office,111,"John, sit down, sit down. Well, what is the D...","President Nixon; John W, Dean, III; H. R. (""Bo...","MARCH 20, 1973",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,37
4,E - 72,421-018,1973-03-21,Old Executive Office Building Office,36,[Unintelligible] to this morning. That's wher...,"President Nixon; John W. Dean, III; H. R. (""Bo...","MARCH 20, 1973",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,38


Add in pickled files try and run this tomorrow. 

## Function to Create SubDocuments
Tokenized every fifteen sentences

In [4]:
def transcript_to_doc(transcript):
    """Turns transcript """
    s = sent_tokenize(transcript) #seperates transcript into sentences
    s = [x for x in s if len(x.split())>1] #make a list comprehension of sentences, sentence greater than one

    s_fifteen = [] #empty list, meant to group each 15 sentences together
    new_string = '' 
    counter = 0
    for sent in s:
        new_string += ' ' #this space seperates the end of sentences, so you dont get this (example.example)
        new_string += sent
        counter += 1
        if counter % 15 == 0: #this should allow break condition, (less than 15 sentences) to be grouped as a final sentence
            s_fifteen.append(new_string) #append string into s_15
            new_string = '' 

    s_fifteen.append(new_string)
            
    return s_fifteen

## Total Transcript List

### List of tokenized transcripts, ordered by transcript

In [5]:
list_word = []
for trans in wg_df['Nixon Words']:
    list_word.append(transcript_to_doc(trans))

NameError: name 'sent_tokenize' is not defined

### Tokenized transcripts in one list


In [ ]:
total_word = []
for sub_list in list_word: 
    for item in sub_list:
        total_word.append(item)

## List of Stop Words 

In [ ]:
my_stop_words = stopwords.words('english')
new_stops = ['unintelligible', 'uh', 'us', 'yeah', 'da', 'right',  'say', 'ready', 'think'
             'well', 'let', 'would', 'see' , 'got', 'get', 'could', 'yes', 'john', 'pause'
            'an', 'la', 'oh', 'says', 'ah', 'ah ah', 'like', 'really', 'going',
            'go', 'gonna', 'know', 'uhh', 'well', 'think', 'thing', 'th', 'ya', 'um', 'hmm', 'hum',
            'come', 'came', 'need', 'five', 'minutes', 'pause', 'sort', 'feel' ]
my_stop_words.extend(new_stops)

## Corex 

### Count Vectorizer Model

In [ ]:
sub_docs = pd.Series(total_word)

In [ ]:
vectorizer = CountVectorizer(max_features=20000,
                             stop_words= my_stop_words, 
                             token_pattern= '(?u)\\b\\w\\w+\\b',  #"\\b[a-z][a-z]+\\b" for just words
                             #max_df = .9, #originally 1, ignore words that appear too often 
                             #min_df = .05, #ignore words that don't appear often enough
                             binary=True

In [ ]:
doc_word = vectorizer.fit_transform(sub_docs)
words = list(np.asarray(vectorizer.get_feature_names()))

### Corex Model

In [ ]:
topic_model_anch = ct.Corex(n_hidden=14, words=words, seed=42)
topic_model_anch.fit(doc_word, words=words, docs=sub_docs,
               anchors=[['money', 'dollars', 'million', 'thousand', 'cash', 'hunt'], #raised
                       ['national', 'security', 'president', 'privileged'],
                       ['house','white','jury','grand','staff','members',],
                       ['executive','privilege','trial'],
                       ['investigation','prosecutor','attorney','general','charge'],
                       ['dean', 'told', 'ehrlichman', 'petersen', 'blackmail'],
                       ['ervin', 'senate', 'committee', 'welcome'],
                       ['pr', 'indicates', 'publicly'],
                       ['plea', 'fifth', 'amendment' ],
                       ['god', 'damn', 'son', 'bitch', 'hell'],
                       ['obstruction', 'justice'],
                       ['segretti', 'vulnerability'],
                       ['fbi', 'gray', 'destroy', 'documents'],
                        ['ellsberg', 'psychiatrist']        
                       ], anchor_strength=2)

In [ ]:
topics = topic_model_anch.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ', '.join(topic_words))

In [ ]:
predictions_anch = pd.DataFrame(topic_model_anch.predict(doc_word), columns=['topic'+str(i) for i in range(14)])

## Find Transcript by Date 

### Find Top Docs

In [ ]:
def find_top_docs(date, topic_num, option):
    
    #change date into datetime format (Y-m-d)
    dt_date = datetime.strptime(date, '%Y-%m-%d')
    
    #Filter dataframe by date
    mask = proba_df['Dates'] == dt_date
    subset_df = proba_df[mask]
    
    #organize dataframe by highest probability
    organized_subset = subset_df.sort_values(by = topic_num, ascending=False)
    
    #return Sub_Doc row, depending on the option
    return (organized_subset.iloc[option, 7], organized_subset.iloc[option, :]) #other idea
    #return organized_subset.head()


### Recommend Transcript by Date

In [6]:
def recommend_transcript_by_date(date, topic_num, option):
    
    #get subdocument transcript for requested topic
    exact_sub_doc, meta_sub_doc = find_top_docs(date, topic_num, option) #other idea


    mask = vf_df['Cassette Number'] == meta_sub_doc.loc['Cassette Number'] #mask to find matching cassette number
    text = vf_df[mask].iloc[0, -2] # relevant transcript matched with cassette number 
    
   
    begin_index = 15
    end_index = -15

    
    print('Cassette Number:  ' + vf_df[mask].iloc[0, 0]+ '\n')
    
    try: 
        result = re.findall(exact_sub_doc[:begin_index] + '[.\s\S]*' + exact_sub_doc[end_index:], text) 
        return print('PRESIDENT:'+ result[0]) 
    except IndexError:
        begin_index-=5
        
    try: 
        result = re.findall(exact_sub_doc[:begin_index] + '[.\s\S]*' + exact_sub_doc[end_index:], text) 
        return print('PRESIDENT:'+ result[0]) 
    except IndexError:
        end_index+=5
        pass
    
    try: 
        result = re.findall(exact_sub_doc[:begin_index] + '[.\s\S]*' + exact_sub_doc[end_index:], text) 
        return print('PRESIDENT:'+ result[0]) 
    except IndexError:
        begin_index-=5
        pass
    
    try: 
        result = re.findall(exact_sub_doc[:begin_index] + '[.\s\S]*' + exact_sub_doc[end_index:], text) 
        return print('PRESIDENT:'+ result[0]) 
    except IndexError:
        end_index+=5
        return print('Uh oh, someone erased the tape!')
        